In [9]:
import os
import numpy as np
import tensorflow as tf
import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, UpSampling2D, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import img_to_array, load_img
import warnings
from PIL import Image
warnings.filterwarnings('ignore')

In [2]:
path_original = '256X'
path_blurred = 'Zgaussowane'

# Funkcja do ładowania i przetwarzania obrazów
def load_images(path, size=(256, 256)):
    images = []
    for filename in sorted(os.listdir(path)):
        img_path = os.path.join(path, filename)
        img = load_img(img_path, target_size=size, color_mode='rgb')
        img = img_to_array(img) / 255.0  # Normalizacja obrazów do zakresu [0, 1]
        images.append(img)
    return np.array(images)

In [3]:
if os.path.isdir('256X/.ipynb_checkpoints'):
    shutil.rmtree('256X/.ipynb_checkpoints')
if os.path.isdir('Zgaussowane/.ipynb_checkpoints'):
    shutil.rmtree('Zgaussowane/.ipynb_checkpoints')

In [4]:
original_images = load_images(path_original)
blurred_images = load_images(path_blurred)

print(f'Liczba załadowanych oryginalnych obrazów: {len(original_images)}')
print(f'Liczba załadowanych zblurowanych obrazów: {len(blurred_images)}')

Liczba załadowanych oryginalnych obrazów: 57
Liczba załadowanych zblurowanych obrazów: 57


In [5]:
def build_model(input_shape):
    model = Sequential()
    model.add(Input(shape=input_shape))
    model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
    model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
    model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
    # model.add(UpSampling2D((2, 2)))
    model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
    model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
    # odel.add(UpSampling2D((2, 2)))
    model.add(Conv2D(3, (3, 3), activation='sigmoid', padding='same'))
    
    model.compile(optimizer=Adam(learning_rate=0.001), loss='mean_squared_error')
    return model

input_shape = (256, 256, 3)
model = build_model(input_shape)
model.summary()

2024-06-05 18:15:19.417643: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-06-05 18:15:19.857235: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2251] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 256, 256, 64)   │         1,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 256, 256, 64)   │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 256, 256, 128)  │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 256, 256, 128)  │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 256, 256, 64)   │        73,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 256, 256, 3)    │         1,731 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 335,683 (1.28 MB)

 Trainable params: 335,683 (1.28 MB)

 Non-trainable params: 0 (0.00 B)

In [6]:
from sklearn.model_selection import train_test_split

# Podział danych na treningowe i walidacyjne
X_train, X_val, y_train, y_val = train_test_split(blurred_images, original_images, random_state=42)

# Trenowanie modelu
history = model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=20, batch_size=8)

# Zapisywanie modelu
# model.save('deblur_cnn_model.h5')
keras.saving.save_model(model, 'deblur_cnn_model_redux.keras')

Epoch 1/20
6/6 ━━━━━━━━━━━━━━━━━━━━ 48s 8s/step - loss: 0.0833 - val_loss: 0.0683
Epoch 2/20
6/6 ━━━━━━━━━━━━━━━━━━━━ 86s 9s/step - loss: 0.0582 - val_loss: 0.0690
Epoch 3/20
6/6 ━━━━━━━━━━━━━━━━━━━━ 56s 9s/step - loss: 0.0616 - val_loss: 0.0479
Epoch 4/20
6/6 ━━━━━━━━━━━━━━━━━━━━ 81s 9s/step - loss: 0.0425 - val_loss: 0.0500
Epoch 5/20
6/6 ━━━━━━━━━━━━━━━━━━━━ 52s 9s/step - loss: 0.0320 - val_loss: 0.0295
Epoch 6/20
6/6 ━━━━━━━━━━━━━━━━━━━━ 50s 8s/step - loss: 0.0218 - val_loss: 0.0216
Epoch 7/20
6/6 ━━━━━━━━━━━━━━━━━━━━ 50s 8s/step - loss: 0.0190 - val_loss: 0.0152
Epoch 8/20
6/6 ━━━━━━━━━━━━━━━━━━━━ 47s 8s/step - loss: 0.0121 - val_loss: 0.0137
Epoch 9/20
6/6 ━━━━━━━━━━━━━━━━━━━━ 44s 7s/step - loss: 0.0114 - val_loss: 0.0128
Epoch 10/20
6/6 ━━━━━━━━━━━━━━━━━━━━ 44s 7s/step - loss: 0.0109 - val_loss: 0.0107
Epoch 11/20
6/6 ━━━━━━━━━━━━━━━━━━━━ 42s 7s/step - loss: 0.0107 - val_loss: 0.0103
Epoch 12/20
6/6 ━━━━━━━━━━━━━━━━━━━━ 84s 8s/step - loss: 0.0083 - val_loss: 0.0093
Epoch 13/20
6

NameError: name 'keras' is not defined

In [12]:
restored_image = model.predict(np.expand_dims(X_val[1], axis=0))

restored_image = restored_image[0] * 255
restored_image = restored_image.astype('uint8')

restored_image_pil = Image.fromarray(restored_image)
restored_image_pil.save('Restored/Coloseum.jpg')

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 434ms/step
